# Preamble

In [1]:
# Imports
## General
import numpy as np
import pickle as pkl
import json
import warnings
warnings.filterwarnings('ignore')
import gc

## Data manipulation and preprocessing
import pandas as pd
import boto3
from sklearn.preprocessing import StandardScaler, FunctionTransformer,LabelEncoder, MinMaxScaler,OrdinalEncoder
#import missingno as msno
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, MissingIndicator
from sklearn.ensemble import HistGradientBoostingRegressor
from scipy import sparse
from tensorflow.keras.layers.experimental.preprocessing import StringLookup, Normalization

## Pre-processing functions
def criteo_log_transform(z):
    if z>2:
        return np.log(z)**2
    else:
        return z
def map_criteo_log_transform(x):
    return x.map(criteo_log_transform)

## Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Image

## Modelling
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
from fastFM import sgd
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
import tensorflow as tf
from deepctr.models.fnn import FNN
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from deepctr.models.pnn import PNN
from deepctr.models.wdl import WDL
from deepctr.models.deepfm import DeepFM
from deepctr.models.autoint import AutoInt
from deepctr.models.fgcnn import FGCNN

# Introduction

As outlined in the Feasibility secition of my Milestone 3 submission, I have segmented the work over the next few weeks according to the following timeline

![](figures/timeline.png)

Below I proceed by structuring my response according to the following sections
- Model Specific Research
- Data exploration, preprocessing and environment setup
- Model Replication and Experimentation

# Model Specific Research

*__Fill out details from readings__*

On the basis of my reasearch and published results, I have chosen to investigate the following models:

__Shallow__
- Logistic Regression
- Factorization Machines

__Deep__
- Factorization Supported Neural Networks
- Product Based Neural Networks
- Wide and Deep
- DeepFM
- Feature Generation by Convolutional Neural Networks
- Automatic Feature Interaction (AutoInt)

# Data Exploration, Caching, Preprocessing pipeline

## Data Exploration

## Load benchmark datasets as tf.datasets

In [2]:
def list_files_in_folder(bucket_name, folder_name):
    out = []
    # Create a Boto3 client for S3
    s3_client = boto3.client('s3')
 
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)
 
 
    if 'Contents' in response:
    
        for obj in response['Contents']:
            out.append(obj['Key'])
        out = out[1:]
        return out
    else:
        print("Folder is empty.")

### KDD12

In [3]:
# get avazu file names
bucket_name = 'mlds-final-project-bucket'
folder_name = 'kdd12/train/train_split/'
keys = list_files_in_folder(bucket_name, folder_name)
filenames = [f"s3://{bucket_name}/{key}" for key in keys]

In [4]:
# Load the kdd12_train dataset
kdd12_train = tf.data.experimental.make_csv_dataset(
    filenames,
    batch_size=1,
    field_delim=',',
    header=True,
    column_defaults=['int32','int32','string','string','string','int32','int32','string','string','string','string','string'],
    label_name='click',
    num_epochs=1,
    shuffle=False,
    #shuffle_buffer_size=100,
    #shuffle_seed=42,
    #ignore_errors=True
)

2024-08-03 14:17:49.884443: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-08-03 14:17:49.887664: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-03 14:17:49.895763: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
# get avazu file names
bucket_name = 'mlds-final-project-bucket'
folder_name = 'kdd12/train/test_split/'
keys = list_files_in_folder(bucket_name, folder_name)
filenames = [f"s3://{bucket_name}/{key}" for key in keys]

In [6]:
# Load the kdd12_train dataset
kdd12_val = tf.data.experimental.make_csv_dataset(
    filenames,
    batch_size=1,
    field_delim=',',
    header=True,
    column_defaults=['int32','int32','string','string','string','int32','int32','string','string','string','string','string'],
    label_name='click',
    num_epochs=1,
    shuffle=False,
    #shuffle_buffer_size=100,
    #shuffle_seed=42,
    #ignore_errors=True
)

### Avazu

In [7]:
# get avazu file names
bucket_name = 'mlds-final-project-bucket'
folder_name = 'avazu/train/train_split/'
keys = list_files_in_folder(bucket_name, folder_name)
filenames = [f"s3://{bucket_name}/{key}" for key in keys]

In [8]:
column_defaults=[
    'string',
    'int32',
    'int32',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string'
]
avazu_train = tf.data.experimental.make_csv_dataset(
    filenames,
    batch_size=1,
    field_delim=',',
    header=True,
    column_defaults=column_defaults,
    label_name='click',
    num_epochs=1,
    shuffle=False,
    #shuffle_buffer_size=100,
    #shuffle_seed=42,
    #ignore_errors=True
)

In [9]:
# get avazu file names
bucket_name = 'mlds-final-project-bucket'
folder_name = 'avazu/train/test_split/'
keys = list_files_in_folder(bucket_name, folder_name)
filenames = [f"s3://{bucket_name}/{key}" for key in keys]

In [10]:
column_defaults=[
    'string',
    'int32',
    'int32',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string',
    'string'
]

# Load the Avazu train dataset
avazu_val = tf.data.experimental.make_csv_dataset(
    filenames,
    batch_size=1,
    field_delim=',',
    header=True,
    column_defaults=column_defaults,
    label_name='click',
    num_epochs=1,
    shuffle=False,
    #shuffle_buffer_size=100,
    #shuffle_seed=42,
    #ignore_errors=True
)

In [11]:
# Drop ID variable
def drop_id(features, label):
    out_features = features.copy()
    del out_features['id']
    return out_features, label

In [12]:
avazu_train = avazu_train.map(drop_id)
avazu_val = avazu_val.map(drop_id)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


### Criteo

In [13]:
# get criteo file names
bucket_name = 'mlds-final-project-bucket'
folder_name = 'dac/train/train_split/'
keys = list_files_in_folder(bucket_name, folder_name)
filenames = [f"s3://{bucket_name}/{key}" for key in keys]

In [14]:
# Load the Criteo train dataset

column_defaults = ['int32'] + 13*['int32'] + 26*['string']

criteo_train = tf.data.experimental.make_csv_dataset(
    filenames,
    batch_size=1,
    field_delim=',',
    header=True,
    label_name='click',
    column_defaults=column_defaults,
    num_epochs=1,
    shuffle=False,
    ignore_errors=True
)

In [15]:
# get criteo file names
bucket_name = 'mlds-final-project-bucket'
folder_name = 'dac/train/test_split/'
keys = list_files_in_folder(bucket_name, folder_name)
filenames = [f"s3://{bucket_name}/{key}" for key in keys]

In [16]:
# Load the Criteo train dataset

column_defaults = ['int32'] + 13*['int32'] + 26*['string']

criteo_val = tf.data.experimental.make_csv_dataset(
    filenames,
    batch_size=1,
    field_delim=',',
    header=True,
    label_name='click',
    column_defaults=column_defaults,
    num_epochs=1,
    shuffle=False,
    ignore_errors=True
)

I first perform an exploratory data analysis to show the type of preprocessing that needs to be made. In order to simplify the data processing requirements, I do this on the basis of $n=10000$ row samples of the data.

## Preprocessing

Below I construct the preprocessing pipeline for the tensorflow datasets on the basis of the exploration functions above. I proceed by:

- Enconding the sparse catecorically features using the Ordinal Encoders specified above.
- Standardizing the numerical features unsing the Scalers and Transformer specified above.
- Generating the feature columns, as specified in the [DeepCTR Quickstart Guide](https://deepctr-doc.readthedocs.io/en/latest/Quick-Start.html#step-3-generate-feature-columns)

In [17]:
## Create lists of categorical colums for each dataset
kdd12_categorical_columns = [
    'DisplayURL',
    'AdID',
    'AdvertiserID',
    'QueryID',
    'KeywordID',
    'TitleID',
    'DescriptionID',
    'UserID'
]

avazu_categorical_columns = [
    'C1',
    'banner_pos',
    'site_id',
    'site_domain',
    'site_category',
    'app_id',
    'app_domain',
    'app_category',
    'device_id',
    'device_ip',
    'device_model',
    'device_type',
    'device_conn_type',
    'C14',
    'C15',
    'C16',
    'C17',
    'C18',
    'C19',
    'C20',
    'C21'
]

criteo_numerical_columns = [f'int_{i}' for i in np.arange(1,14)]
criteo_categorical_columns = [f'cat_{i}' for i in np.arange(1,27)]

### Categorical Variable Encoding

In [18]:
# Import categorical feature mappings and define stringloohup objects for each dataset
kdd12_stringlookups = {}
for field in kdd12_categorical_columns:
    df = pd.read_csv(f'./data/kdd12/categorical_value_counts/{field}.csv')
    vocab = [elem.encode() for elem in df['field'].astype(str).to_list()]
    lookup = StringLookup(vocabulary=vocab, mask_token=None)
    kdd12_stringlookups.update({field:lookup})

avazu_stringlookups = {}
for field in avazu_categorical_columns:
    df = pd.read_csv(f'./data/avazu/categorical_value_counts/{field}.csv')
    vocab = [elem.encode() for elem in df['field'].astype(str).to_list()]
    lookup = StringLookup(vocabulary=vocab, mask_token=None)
    avazu_stringlookups.update({field:lookup})

criteo_stringlookups = {}
for field in criteo_categorical_columns:
    df = pd.read_csv(f'./data/criteo/categorical_value_counts/{field}.csv')
    vocab = [elem.encode() for elem in df['field'].astype(str).to_list()]
    lookup = StringLookup(vocabulary=vocab, mask_token=None)
    criteo_stringlookups.update({field:lookup})

In [19]:
# Define categorical encoding function
@tf.function
def kdd12_categorical_encoding(features,label):
    # Create copy of features, because modifying inputs causes a ValueError
    out_features = features.copy()
    # Iteratively map the categical feature columns using the corresponging Lookup layer
    for f in kdd12_categorical_columns:
        lookup = kdd12_stringlookups[f]
        out_features[f.lower()] = lookup(features[f.lower()])
    return out_features, label

@tf.function
def avazu_categorical_encoding(features,label):
    # Create copy of features, because modifying inputs causes a ValueError
    out_features = features.copy()
    # Iteratively map the categical feature columns using the corresponging Lookup layer
    for f in avazu_categorical_columns:
        lookup = avazu_stringlookups[f]
        out_features[f] = lookup(features[f])
    return out_features, label

@tf.function
def criteo_categorical_encoding(features,label):
    # Create copy of features
    out_features = features.copy()
    # Iteratively map the categorical feature columns using the corresponding Lookup layer
    for f in criteo_categorical_columns:
        lookup = criteo_stringlookups[f]
        out_features[f] = lookup(features[f])
    return out_features, label

In [20]:
# Apply mapping over each dataset

kdd12_train_encoded = kdd12_train.map(kdd12_categorical_encoding)
avazu_train_encoded = avazu_train.map(avazu_categorical_encoding)
criteo_train_encoded = criteo_train.map(criteo_categorical_encoding)

kdd12_val_encoded = kdd12_val.map(kdd12_categorical_encoding)
avazu_val_encoded = avazu_val.map(avazu_categorical_encoding)
criteo_val_encoded = criteo_val.map(criteo_categorical_encoding)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

### Numerical Variable Scaling

In [21]:
# Define numerical feature columns
kdd12_numerical_columns = [
    'Depth',
    'Position',
    'Impression'
]

avazu_numerical_columns = [
    'hour'
]

criteo_numerical_columns = [f'int_{i}' for i in np.arange(1,14)]

In [22]:
# Extract scaler dicts for all datasets
dist_stats = pd.read_csv('./data/kdd12/means_variances.csv')
kdd12_scalers = {}
for i in range(len(dist_stats)):
    field = dist_stats['field'][i]
    mean = dist_stats['mean'][i]
    variance = dist_stats['variance'][i]
    scaler = Normalization(mean=mean, variance=variance)
    kdd12_scalers.update({field:scaler})

dist_stats = pd.read_csv('./data/avazu/means_variances.csv')
avazu_scalers = {}
for i in range(len(dist_stats)):
    field = dist_stats['field'][i]
    mean = dist_stats['mean'][i]
    variance = dist_stats['variance'][i]
    scaler = Normalization(mean=mean, variance=variance)
    avazu_scalers.update({field:scaler})

dist_stats = pd.read_csv('./data/criteo/means_variances.csv')
criteo_scalers = {}
for i in range(len(dist_stats)):
    field = dist_stats['field'][i]
    mean = dist_stats['mean'][i]
    variance = dist_stats['variance'][i]
    scaler = Normalization(mean=mean, variance=variance)
    criteo_scalers.update({field:scaler})

In [23]:
# Define scaler functions for all datasets

@tf.function
def kdd12_numerical_scaling(features,label):
    out_features = features.copy()
    for f in kdd12_numerical_columns:
        scaler = kdd12_scalers[f]
        out_features[f.lower()] = scaler(features[f.lower()])
    return out_features, label

@tf.function
def avazu_numerical_scaling(features,label):
    out_features = features.copy()
    for f in avazu_numerical_columns:
        scaler = avazu_scalers[f]
        out_features[f] = scaler(features[f])
    return out_features, label

@tf.function
def criteo_numerical_scaling(features,label):
    out_features = features.copy()
    for f in criteo_numerical_columns:
        scaler = criteo_scalers[f]
        out_features[f] = scaler(features[f])
    return out_features, label

In [24]:
# Apply the numerical scaling to all datasets
kdd12_train_scaled = kdd12_train_encoded.map(kdd12_numerical_scaling)
avazu_train_scaled = avazu_train_encoded.map(avazu_numerical_scaling)
criteo_train_scaled = criteo_train_encoded.map(criteo_numerical_scaling)

kdd12_val_scaled = kdd12_val_encoded.map(kdd12_numerical_scaling)
avazu_val_scaled = avazu_val_encoded.map(avazu_numerical_scaling)
criteo_val_scaled = criteo_val_encoded.map(criteo_numerical_scaling)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [25]:
# Print the element specs
print('KDD12:')
print(kdd12_train_scaled.element_spec)
print()
print('Avazu:')
print(avazu_train_scaled.element_spec)
print()
print('Criteo:')
print(criteo_train_scaled.element_spec)

KDD12:
(OrderedDict([('impression', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)), ('displayurl', TensorSpec(shape=(None,), dtype=tf.int64, name=None)), ('adid', TensorSpec(shape=(None,), dtype=tf.int64, name=None)), ('advertiserid', TensorSpec(shape=(None,), dtype=tf.int64, name=None)), ('depth', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)), ('position', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)), ('keywordid', TensorSpec(shape=(None,), dtype=tf.int64, name=None)), ('titleid', TensorSpec(shape=(None,), dtype=tf.int64, name=None)), ('descriptionid', TensorSpec(shape=(None,), dtype=tf.int64, name=None)), ('queryid', TensorSpec(shape=(None,), dtype=tf.int64, name=None)), ('userid', TensorSpec(shape=(None,), dtype=tf.int64, name=None))]), TensorSpec(shape=(None,), dtype=tf.int32, name=None))

Avazu:
(OrderedDict([('hour', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)), ('C1', TensorSpec(shape=(None,), dtype=tf.int64, name=None)), ('banner

In [26]:
# Apply log transformation to criteo datase

@tf.function
def criteo_log_scaling(features,label):
    out_features = features.copy()
    for f in criteo_numerical_columns:
        x = features[f]
        out_features[f] = tf.where(x>2,tf.math.square(tf.math.log(x)),x)
    return out_features, label

In [27]:
criteo_train_scaled = criteo_train_scaled.map(criteo_log_scaling)
criteo_val_scaled = criteo_val_scaled.map(criteo_log_scaling)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [28]:
# Define model inputs
kdd12_train_model_input = kdd12_train_scaled.take(157440).batch(256).prefetch(tf.data.AUTOTUNE)
avazu_train_model_input = avazu_train_scaled.take(157440).batch(256).prefetch(tf.data.AUTOTUNE)
criteo_train_model_input = criteo_train_scaled.take(157440).batch(256).prefetch(tf.data.AUTOTUNE)

kdd12_val_model_input = kdd12_val_scaled.take(39360).batch(256).prefetch(tf.data.AUTOTUNE)
avazu_val_model_input = avazu_val_scaled.take(39360).batch(256).prefetch(tf.data.AUTOTUNE)
criteo_val_model_input = criteo_val_scaled.take(39360).batch(256).prefetch(tf.data.AUTOTUNE)

### Define Feature columns

In [29]:
## Define feature mappings
kdd12_fixlen_feature_columns = [SparseFeat(feat.lower(), vocabulary_size=len(kdd12_stringlookups[feat].get_vocabulary()), embedding_dim=4) for feat in kdd12_categorical_columns]\
+ [DenseFeat(feat.lower(),1) for feat in kdd12_numerical_columns]
avazu_fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=len(avazu_stringlookups[feat].get_vocabulary()), embedding_dim=4) for feat in avazu_categorical_columns]\
+ [DenseFeat(feat,1) for feat in avazu_numerical_columns]
criteo_fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=len(criteo_stringlookups[feat].get_vocabulary()), embedding_dim=4) for feat in criteo_categorical_columns]\
+ [DenseFeat(feat,1) for feat in criteo_numerical_columns]

## Generate the dnn and linear feature columns
kdd12_dnn_feature_columns = kdd12_fixlen_feature_columns
kdd12_linear_feature_columns = kdd12_fixlen_feature_columns

avazu_dnn_feature_columns = avazu_fixlen_feature_columns
avazu_linear_feature_columns = avazu_fixlen_feature_columns

criteo_dnn_feature_columns = criteo_fixlen_feature_columns
criteo_linear_feature_columns = criteo_fixlen_feature_columns

# Model Replication and Experimentation

## Shallow Models

In order to attain representitive results from beyond the realm of deep learning, below I evaluate two of the most poplarly used shallow models for CTR prediction:

- [Logistic regression](https://www.tensorflow.org/guide/core/logistic_regression_core)
- Factorization machines

### Logistic Regression

In [3]:
%run -i scripts/modelling/fit_lr_models.py
%run -i scripts/modelling/score_lr_models.py
%run -i scripts/modelling/save_lr_models.py

KDD12:
Log loss: 0.162493229115702
ROC AUC: 0.69918173566772
Accuracy: 0.95825

Avazu:
Log loss: 0.41220123857803254
ROC AUC: 0.7187724509322894
Accuracy: 0.83205

Criteo:
Log loss: 0.4926044086201233
ROC AUC: 0.7461917357547045
Accuracy: 0.76775


### Factorization Machine

In [4]:
%run -i scripts/modelling/fit_fm_models.py
%run -i scripts/modelling/score_fm_models.py
%run -i scripts/modelling/save_fm_models.py

KDD12:
Log loss: 0.31685987446578695
ROC AUC: 0.5336269490760196
Accuracy: 0.95825

Avazu:
Log loss: 10.974548402513749
ROC AUC: 0.5212335152327499
Accuracy: 0.66185

Criteo:
Log loss: 26.85071959222282
ROC AUC: 0.5000335570469798
Accuracy: 0.25505


## Deep Models

As explained above, I will be exploring the following deep models:

- Factorization Supported Neural Networks
- Product Based Neural Networks
- Wide and Deep
- DeepFM
- Feature Generation by Convolutional Neural Networks
- Automatic Feature Interaction (AutoInt)

### Factorization-Machine Supported Neural Networks

In [30]:
# Compile the models
kdd12_fnn_model = FNN(kdd12_linear_feature_columns, kdd12_dnn_feature_columns, task='binary')
kdd12_fnn_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

avazu_fnn_model = FNN(avazu_linear_feature_columns, avazu_dnn_feature_columns, task='binary')
avazu_fnn_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

criteo_fnn_model = FNN(criteo_linear_feature_columns, criteo_dnn_feature_columns, task='binary')
criteo_fnn_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [31]:
# Define the CSVLogger callbacks
kdd12_fnn_csvLogger = CSVLogger('logs/kdd12_fnn.csv')
avazu_fnn_csvLogger = CSVLogger('logs/avazu_fnn.csv')
criteo_fnn_csvLogger = CSVLogger('logs/criteo_fnn.csv')

In [32]:
# Define the model saving checkpoints
kdd12_fnn_modelCheckpoint = ModelCheckpoint('models/fnn/kdd12_fnn_model.keras',save_best_only=True)
avazu_fnn_modelCheckpoint = ModelCheckpoint('models/fnn/avazu_fnn_model.keras',save_best_only=True)
criteo_fnn_modelCheckpoint = ModelCheckpoint('models/fnn/criteo_fnn_model.keras',save_best_only=True)

In [33]:
# Fit the models
kdd12_fnn_history = kdd12_fnn_model.fit(kdd12_train_model_input, validation_data=kdd12_val_model_input, batch_size=256, epochs=15, callbacks=[kdd12_fnn_csvLogger,kdd12_fnn_modelCheckpoint])
avazu_fnn_history = avazu_fnn_model.fit(avazu_train_model_input, validation_data=avazu_val_model_input, batch_size=256, epochs=15, callbacks=[avazu_fnn_csvLogger,avazu_fnn_modelCheckpoint])
criteo_fnn_history = criteo_fnn_model.fit(criteo_train_model_input, validation_data=criteo_val_model_input, batch_size=256, epochs=15, callbacks=[criteo_fnn_csvLogger,criteo_fnn_modelCheckpoint])

2024-08-03 14:25:19.278989: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-08-03 14:25:19.302715: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2899990000 Hz


Epoch 1/15
615/615 [==============================] - 154s 246ms/step - loss: 0.2364 - binary_crossentropy: 0.2355 - val_loss: 0.1402 - val_binary_crossentropy: 0.1355
Epoch 2/15
615/615 [==============================] - 149s 242ms/step - loss: 0.1208 - binary_crossentropy: 0.1149 - val_loss: 0.1829 - val_binary_crossentropy: 0.1738
Epoch 3/15
615/615 [==============================] - 150s 243ms/step - loss: -0.0030 - binary_crossentropy: -0.0114 - val_loss: 0.2515 - val_binary_crossentropy: 0.2450
Epoch 4/15
615/615 [==============================] - 151s 245ms/step - loss: -0.0348 - binary_crossentropy: -0.0411 - val_loss: 0.2413 - val_binary_crossentropy: 0.2361
Epoch 5/15
615/615 [==============================] - 150s 244ms/step - loss: -0.0454 - binary_crossentropy: -0.0505 - val_loss: 0.2962 - val_binary_crossentropy: 0.2916
Epoch 6/15
615/615 [==============================] - 151s 245ms/step - loss: -0.0495 - binary_crossentropy: -0.0539 - val_loss: 0.3381 - val_binary_cross

KeyboardInterrupt: 

In [ ]:
# Read in the history logs
kdd12_fnn_history = pd.read_csv('logs/kdd12_fnn.csv')
avazu_fnn_history = pd.read_csv('logs/avazu_fnn.csv')
criteo_fnn_history = pd.read_csv('logs/criteo_fnn.csv')

# Plot the training history
plt.plot(kdd12_fnn_history['binary_crossentropy'])
plt.plot(kdd12_fnn_history['val_binary_crossentropy'])
plt.title('kdd12_fnn_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(avazu_fnn_history['binary_crossentropy'])
plt.plot(avazu_fnn_history['val_binary_crossentropy'])
plt.title('avazu_fnn_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(criteo_fnn_history['binary_crossentropy'])
plt.plot(criteo_fnn_history['val_binary_crossentropy'])
plt.title('criteo_fnn_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Score the models
kdd12_fnn_score = kdd12_fnn_model.evaluate(kdd12_val_model_input, batch_size=256)
avazu_fnn_score = avazu_fnn_model.evaluate(avazu_val_model_input, batch_size=256)
criteo_fnn_score = criteo_fnn_model.evaluate(criteo_val_model_input, batch_size=256)

### Product Based Neural Networks

In [ ]:
# Define the CSVLogger callbacks
kdd12_pnn_csvLogger = CSVLogger('logs/kdd12_pnn.csv')
avazu_pnn_csvLogger = CSVLogger('logs/avazu_pnn.csv')
criteo_pnn_csvLogger = CSVLogger('logs/criteo_pnn.csv')

In [ ]:
# Define the model saving checkpoints
kdd12_pnn_modelCheckpoint = ModelCheckpoint('models/pnn/kdd12_pnn_model.keras',save_best_only=True)
avazu_pnn_modelCheckpoint = ModelCheckpoint('models/pnn/avazu_pnn_model.keras',save_best_only=True)
criteo_pnn_modelCheckpoint = ModelCheckpoint('models/pnn/criteo_pnn_model.keras',save_best_only=True)

In [ ]:
# Compile the PNN models
kdd12_pnn_model = PNN(kdd12_dnn_feature_columns, task='binary')
kdd12_pnn_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

avazu_pnn_model = PNN(avazu_dnn_feature_columns, task='binary')
avazu_pnn_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

criteo_pnn_model = PNN(criteo_dnn_feature_columns, task='binary')
criteo_pnn_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

In [ ]:
# Fit the models
kdd12_pnn_history = kdd12_pnn_model.fit(
    kdd12_train_model_input, 
    validation_data=kdd12_val_model_input,
    batch_size=256, 
    epochs=40, 
    callbacks=[kdd12_pnn_csvLogger,kdd12_pnn_modelCheckpoint]
)
avazu_pnn_history = avazu_pnn_model.fit(
    avazu_train_model_input,
    validation_data=avazu_val_model_input,
    batch_size=256, 
    epochs=40,
    callbacks=[avazu_pnn_csvLogger,avazu_pnn_modelCheckpoint]
)
criteo_pnn_history = criteo_pnn_model.fit(
    criteo_train_model_input,
    validation_data=criteo_val_model_input,
    batch_size=256, 
    epochs=40,
    callbacks=[criteo_pnn_csvLogger,criteo_pnn_modelCheckpoint]
)

In [ ]:
# Read in the history logs
kdd12_history = pd.read_csv('logs/kdd12_pnn.csv')
avazu_history = pd.read_csv('logs/avazu_pnn.csv')
criteo_history = pd.read_csv('logs/criteo_pnn.csv')

# Plot the training history
plt.plot(kdd12_history['binary_crossentropy'])
plt.plot(kdd12_history['val_binary_crossentropy'])
plt.title('kdd12_pnn_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(avazu_history['binary_crossentropy'])
plt.plot(avazu_history['val_binary_crossentropy'])
plt.title('avazu_pnn_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(criteo_history['binary_crossentropy'])
plt.plot(criteo_history['val_binary_crossentropy'])
plt.title('criteo_pnn_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### Wide and Deep

In [ ]:
# Compile the WDL models
kdd12_wdl_model = WDL(kdd12_linear_feature_columns, kdd12_dnn_feature_columns, task='binary')
kdd12_wdl_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

avazu_wdl_model = WDL(avazu_linear_feature_columns, avazu_dnn_feature_columns, task='binary')
avazu_wdl_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

criteo_wdl_model = WDL(criteo_linear_feature_columns, criteo_dnn_feature_columns, task='binary')
criteo_wdl_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

In [ ]:
# Define the CSVLogger callbacks
kdd12_wdl_csvLogger = CSVLogger('logs/kdd12_wdl.csv')
avazu_wdl_csvLogger = CSVLogger('logs/avazu_wdl.csv')
criteo_wdl_csvLogger = CSVLogger('logs/criteo_wdl.csv')

In [ ]:
# Define the model saving checkpoints
kdd12_wdl_modelCheckpoint = ModelCheckpoint('models/wdl/kdd12_wdl_model.keras',save_best_only=True)
avazu_wdl_modelCheckpoint = ModelCheckpoint('models/wdl/avazu_wdl_model.keras',save_best_only=True)
criteo_wdl_modelCheckpoint = ModelCheckpoint('models/wdl/criteo_wdl_model.keras',save_best_only=True)

In [ ]:
# Fit the Wide and Deep models
kdd12_wdl_history = kdd12_wdl_model.fit(
    kdd12_train_model_input, 
    validation_data=kdd12_val_model_input, 
    batch_size=256, 
    epochs=15,
    callbacks=[kdd12_wdl_csvLogger,kdd12_wdl_modelCheckpoint]
)
avazu_wdl_history = avazu_wdl_model.fit(
    avazu_train_model_input, 
    validation_data=avazu_val_model_input, 
    batch_size=256, 
    epochs=15,
    callbacks=[avazu_wdl_csvLogger,avazu_wdl_modelCheckpoint]
)
criteo_wdl_history = criteo_wdl_model.fit(
    criteo_train_model_input, 
    validation_data=criteo_val_model_input, 
    batch_size=256, 
    epochs=15,
    callbacks=[criteo_wdl_csvLogger, criteo_wdl_modelCheckpoint]
)

In [ ]:
# Read in the history logs
kdd12_history = pd.read_csv('logs/kdd12_wdl.csv')
avazu_history = pd.read_csv('logs/avazu_wdl.csv')
criteo_history = pd.read_csv('logs/criteo_wdl.csv')

# Plot the training history
plt.plot(kdd12_history['binary_crossentropy'])
plt.plot(kdd12_history['val_binary_crossentropy'])
plt.title('kdd12_wdl_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(avazu_history['binary_crossentropy'])
plt.plot(avazu_history['val_binary_crossentropy'])
plt.title('avazu_wdl_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(criteo_history['binary_crossentropy'])
plt.plot(criteo_history['val_binary_crossentropy'])
plt.title('criteo_wdl_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### DeepFM

In [ ]:
# Compile the DeepFM models
kdd12_dfm_model = DeepFM(kdd12_linear_feature_columns, kdd12_dnn_feature_columns, task='binary')
kdd12_dfm_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

avazu_dfm_model = DeepFM(avazu_linear_feature_columns, avazu_dnn_feature_columns, task='binary')
avazu_dfm_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

criteo_dfm_model = DeepFM(criteo_linear_feature_columns, criteo_dnn_feature_columns, task='binary')
criteo_dfm_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

In [ ]:
# Define the CSVLogger callbacks
kdd12_dfm_csvLogger = CSVLogger('logs/kdd12_dfm.csv')
avazu_dfm_csvLogger = CSVLogger('logs/avazu_dfm.csv')
criteo_dfm_csvLogger = CSVLogger('logs/criteo_dfm.csv')

In [ ]:
# Define the model saving checkpoints
kdd12_dfm_modelCheckpoint = ModelCheckpoint('models/dfm/kdd12_dfm_model.keras',save_best_only=True)
avazu_dfm_modelCheckpoint = ModelCheckpoint('models/dfm/avazu_dfm_model.keras',save_best_only=True)
criteo_dfm_modelCheckpoint = ModelCheckpoint('models/dfm/criteo_dfm_model.keras',save_best_only=True)

In [ ]:
# Fit the DeepFM models
kdd12_dfm_history = kdd12_dfm_model.fit(
    kdd12_train_model_input, 
    validation_data = kdd12_val_model_input, 
    batch_size=256, 
    epochs=15,
    callbacks=[kdd12_dfm_csvLogger,kdd12_wdl_modelCheckpoint]
)
avazu_dfm_history = avazu_dfm_model.fit(
    avazu_train_model_input, 
    validation_data= avazu_val_model_input, 
    batch_size=256, 
    epochs=15,
    callbacks=[avazu_dfm_csvLogger,avazu_wdl_modelCheckpoint]
)
criteo_dfm_history = criteo_dfm_model.fit(
    criteo_train_model_input, 
    validation_data=criteo_val_model_input, 
    batch_size=256, epochs=15,
    callbacks=[criteo_dfm_csvLogger,criteo_dfm_modelCheckpoint]
)

In [ ]:
# Read in the history logs
kdd12_dfm_history = pd.read_csv('logs/kdd12_dfm.csv')
avazu_dfm_history = pd.read_csv('logs/avazu_dfm.csv')
criteo_dfm_history = pd.read_csv('logs/criteo_dfm.csv')

# Plot the training history
plt.plot(kdd12_dfm_history['binary_crossentropy'])
plt.plot(kdd12_dfm_history['val_binary_crossentropy'])
plt.title('kdd12_dfm_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(avazu_dfm_history['binary_crossentropy'])
plt.plot(avazu_dfm_history['val_binary_crossentropy'])
plt.title('avazu_dfm_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(criteo_dfm_history['binary_crossentropy'])
plt.plot(criteo_dfm_history['val_binary_crossentropy'])
plt.title('criteo_dfm_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### Automatic Feature Interaction (AutoInt)

In [ ]:
# Compile the AutoInt Models
kdd12_autoint_model = AutoInt(kdd12_linear_feature_columns, kdd12_dnn_feature_columns, task='binary')
kdd12_autoint_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

avazu_autoint_model = AutoInt(avazu_linear_feature_columns, avazu_dnn_feature_columns, task='binary')
avazu_autoint_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

criteo_autoint_model = AutoInt(criteo_linear_feature_columns, criteo_dnn_feature_columns, task='binary')
criteo_autoint_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

In [ ]:
# Define the AutoInt CSVLogger callbacks
kdd12_autoint_csvLogger = CSVLogger('logs/kdd12_autoint.csv')
avazu_autoint_csvLogger = CSVLogger('logs/avazu_autoint.csv')
criteo_autoint_csvLogger = CSVLogger('logs/criteo_autoint.csv')

In [ ]:
# Define the model saving checkpoints
kdd12_autoint_modelCheckpoint = ModelCheckpoint('models/autoint/kdd12_autoint_model.keras',save_best_only=True)
avazu_autoint_modelCheckpoint = ModelCheckpoint('models/autoint/avazu_autoint_model.keras',save_best_only=True)
criteo_autoint_modelCheckpoint = ModelCheckpoint('models/autoint/criteo_autoint_model.keras',save_best_only=True)

In [ ]:
# Fit the AutoInt models
kdd12_autoint_model.fit(
    kdd12_train_model_input, 
    validation_data= kdd12_val_model_input, 
    batch_size=256, 
    epochs=15, 
    callbacks=[kdd12_autoint_csvLogger, kdd12_autoint_modelCheckpoint]
)
avazu_autoint_model.fit(
    avazu_train_model_input,
    validation_data=avazu_val_model_input,
    batch_size=256, 
    epochs=15, 
    callbacks=[avazu_autoint_csvLogger, avazu_autoint_modelCheckpoint]
)
criteo_autoint_model.fit(
    criteo_train_model_input, 
    validation_data=criteo_val_model_input, 
    batch_size=256, 
    epochs=15,
    callbacks=[criteo_autoint_csvLogger,criteo_autoint_modelCheckpoint]
)

In [ ]:
# Read in the history logs
kdd12_autoint_history = pd.read_csv('logs/kdd12_autoint.csv')
avazu_autoint_history = pd.read_csv('logs/avazu_autoint.csv')
criteo_autoint_history = pd.read_csv('logs/criteo_autoint.csv')

# Plot the training history
plt.plot(kdd12_autoint_history['binary_crossentropy'])
plt.plot(kdd12_autoint_history['val_binary_crossentropy'])
plt.title('kdd12_autoint_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(avazu_autoint_history['binary_crossentropy'])
plt.plot(avazu_autoint_history['val_binary_crossentropy'])
plt.title('avazu_autoint_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(criteo_autoint_history['binary_crossentropy'])
plt.plot(criteo_autoint_history['val_binary_crossentropy'])
plt.title('criteo_autoint_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### Feature Generation by convolutional Neural Network - FGCNN

In [ ]:
# Compile the FGCNN models

#kdd12_fgcnn_model = FGCNN(kdd12_linear_feature_columns, kdd12_dnn_feature_columns, task='binary')
#kdd12_fgcnn_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

avazu_fgcnn_model = FGCNN(
    avazu_linear_feature_columns, 
    avazu_dnn_feature_columns,
    conv_kernel_width=[7,7,7,7],
    conv_filters=[14,16,18,20],
    new_maps=[3,3,3,3],
    dnn_hidden_units=[4096,2048,1024,512,1],# This was different
    task='binary'
)
avazu_fgcnn_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

criteo_fgcnn_model = FGCNN(
    avazu_linear_feature_columns, 
    avazu_dnn_feature_columns,
    conv_kernel_width=[9,9,9,9],
    conv_filters=[38,40,42,44],
    new_maps=[3,3,3,3],
    dnn_hidden_units=[4096,2048,1],# This was different
    task='binary'
)
criteo_fgcnn_model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

In [ ]:
# Define the AutoInt CSVLogger callbacks
kdd12_fgcnn_csvLogger = CSVLogger('logs/kdd12_fgcnn.csv')
avazu_fgcnn_csvLogger = CSVLogger('logs/avazu_fgcnn.csv')
criteo_fgcnn_csvLogger = CSVLogger('logs/criteo_fgcnn.csv')

In [ ]:
# Define the model saving checkpoints
kdd12_fgcnn_modelCheckpoint = ModelCheckpoint('models/fgcnn/kdd12_fgcnn_model.keras',save_best_only=True)
avazu_fgcnn_modelCheckpoint = ModelCheckpoint('models/fgcnn/avazu_fgcnn_model.keras',save_best_only=True)
criteo_fgcnn_modelCheckpoint = ModelCheckpoint('models/fgcnn/criteo_fgcnn_model.keras',save_best_only=True)

In [ ]:
avazu_fgcnn_history = avazu_fgcnn_model.fit(
    avazu_train_model_input,
    validation_data=avazu_val_model_input,
    batch_size=256,
    epochs=15,
    callbacks=[avazu_fgcnn_csvLogger,avazu_fgcnn_modelCheckpoint]
)
criteo_fgcnn_history = criteo_fgcnn_model.fit(
    criteo_train_model_input,
    validation_data=criteo_val_model_input,
    batch_size=256,
    epochs=15,
    callbacks=[criteo_fgcnn_csvLogger,criteo_fgcnn_modelCheckpoint]
)

In [ ]:
# Read in the history logs
#kdd12_history = pd.read_csv('logs/kdd12_wdl.csv')
avazu_history = pd.read_csv('logs/avazu_fgcnn.csv')
criteo_history = pd.read_csv('logs/criteo_fgcnn.csv')

# Plot the training history
'''
plt.plot(kdd12_history['binary_crossentropy'])
plt.plot(kdd12_history['val_binary_crossentropy'])
plt.title('kdd12_wdl_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
'''

plt.plot(avazu_history['binary_crossentropy'])
plt.plot(avazu_history['val_binary_crossentropy'])
plt.title('avazu_fgcnn_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(criteo_history['binary_crossentropy'])
plt.plot(criteo_history['val_binary_crossentropy'])
plt.title('criteo_fgcnn_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()